# Set-up

In [ ]:
import os
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns

# Load data

In [ ]:
features = ["plant_id", "country", 
                "capacity", "capacity_unit",
                "latitude", "longitude", "location_accuracy",
                "ownership_parent_name", "ownership_parent_id",
                "ownership_owner_name", "ownership_owner_id",
                "ownership_operator_name", "ownership_operator_id",
                "sector"]

# start operating year
    

## get filenames of files of interest

In [ ]:
data_folder = "data/asset_level_data/global_energy_monitor/"

files_for_import = os.listdir(data_folder)

# remove files for pipelines and GEM preprocessed data
files_for_import = [file for file in files_for_import if 'Pipeline' not in file and 'preprocessed' not in file]
files_for_import

## extract sector names for files of interest

In [ ]:
# create simple names for each dataset
def extract_sectors(files_for_import): 

    sectors = []
    for file in files_for_import: 
        if "Global" in file: 
        
            start = 'Global-'
            end = '-Tracker'
            
        else:
            start = 'GEM-GGIT-'
            end = '-2023'
        
        sector = file[file.find(start)+len(start):file.rfind(end)].lower().replace('-', '_')
        sectors.append(sector)
    
    return sectors

sectors = extract_sectors(files_for_import)
sectors

## extract sheet names for files of interest

In [ ]:
sheets = []

for sector, file in list(zip(sectors, files_for_import)):
    xl = pd.ExcelFile(data_folder + file)    
    names = xl.sheet_names  # see all sheet names
    
    if 'About' in names: 
        sheets.append(names[1])
    else: 
        sheets.append(names[0])

In [ ]:
dict_files = {}
for sector, file, sheet in list(zip(sectors, files_for_import, sheets)): 
    dict_files[sector] = {'file_name': file, 'sheet_name': sheet}

dict_files['nuclear_power']['file_name']


## LNG Terminals

In [ ]:
# load data
lng_terminals           = pd.read_excel(data_folder + dict_files['lng_terminals']['file_name'], sheet_name = dict_files['lng_terminals']['sheet_name'])

# rename columns
lng_terminals.columns   = lng_terminals.columns.str.lower()

# report pre-cleaning
print(f"LNG Terminals data before cleaning dimensions: {lng_terminals.shape}")

# keep only operating plants
lng_terminals = lng_terminals[lng_terminals.status == "Operating"].reset_index()
print(f"Number of operating plants: {lng_terminals.shape[0]}")

# convert terminal and unitname into one
lng_terminals['unitname'].fillna(value = "", inplace = True)
lng_terminals['asset_name'] = lng_terminals.terminalname + " " + lng_terminals.unitname

# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", "parent", "parent_location", 
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "proposal_year", "construction_start_year", "status", "last_updated"]

cols_to_keep = ['asset_name', 'owner', 'parent', 'parenthqcountry', 
                'capacity', 'country', 'region', 'location', 'latitude', 'longitude', 
                'accuracy', 'startyearearliest', 'proposalyear', 'constructionyear', 'status', 'lastupdated']

lng_terminals = lng_terminals[cols_to_keep]
lng_terminals.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
lng_terminals['sector'] = "LNG terminal"

# look at the final set
lng_terminals



# Nuclear power

In [ ]:
# load data
nuclear_power           = pd.read_excel(data_folder + dict_files['nuclear_power']['file_name'], sheet_name = dict_files['nuclear_power']['sheet_name'])

# rename columns
nuclear_power.columns   = nuclear_power.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Nuclear power data before cleaning dimensions: {nuclear_power.shape}")

# keep only operating plants
nuclear_power = nuclear_power[nuclear_power.status == "operating"].reset_index()
print(f"Number of operating plants: {nuclear_power.shape[0]}")

# convert terminal and unitname into one
nuclear_power['unit_name'].fillna(value = "", inplace = True)
nuclear_power['asset_name'] = nuclear_power.project_name + " " + nuclear_power.unit_name

# convert construction date
nuclear_power['construction_year'] = pd.to_datetime(nuclear_power['construction_start_date']).dt.year

# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", 
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status", "construction_year",  "last_updated"]

cols_to_keep = ['asset_name', 'owner', 
                'capacity_(mw)', 'country', 'region', 'city', 'latitude', 'longitude', 
                'location_accuracy', 'start_year', 'status', 'construction_year', 'date_last_researched']
            

nuclear_power = nuclear_power[cols_to_keep]
nuclear_power.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate column for sector 
nuclear_power["sector"] = "nuclear power plant"

# look at the final set
nuclear_power


# Hydropower

In [ ]:
# load data
hydropower        = pd.read_excel(data_folder + dict_files['hydropower']['file_name'], sheet_name = dict_files['hydropower']['sheet_name'])

# rename columns
hydropower.columns   = hydropower.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Hydropower data before cleaning dimensions: {hydropower.shape}")

# keep only operating plants
hydropower = hydropower[hydropower.status == "operating"].reset_index()
print(f"Number of operating plants: {hydropower.shape[0]}")

# # keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", 
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status"]

cols_to_keep = ['project_name', 'owner', 
                'capacity_(mw)', 'country_1', 'region_1', 'city_1', 'latitude', 'longitude', 
                'location_accuracy', 'start_year', 'status']
    # does not have parent owner info + date last researched

hydropower = hydropower[cols_to_keep]
hydropower.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# # generate column for sector 
hydropower["sector"] = "hydropower plant"

# # look at the final set
hydropower.columns


# Geothermal

In [ ]:
# load data
geothermal_power           = pd.read_excel(data_folder + dict_files['geothermal_power']['file_name'], sheet_name = dict_files['geothermal_power']['sheet_name'])

# rename columns
geothermal_power.columns   = geothermal_power.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Geothermal data before cleaning dimensions: {geothermal_power.shape}")

# keep only operating plants
geothermal_power = geothermal_power[geothermal_power.status == "operating"].reset_index()
print(f"Number of operating plants: {geothermal_power.shape[0]}")

# # convert terminal and unitname into one
geothermal_power['unit_name'].fillna(value = "", inplace = True)
geothermal_power['asset_name'] = geothermal_power.project_name + " " + geothermal_power.unit_name

# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner",  
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status", "last_updated"]

cols_to_keep = ['asset_name', 'owner', 
                'unit_capacity_(mw)', 'country', 'region', 'city', 'latitude', 'longitude', 
                'location_accuracy', 'start_year', 'status', 'date_last_researched']
    # does not have info about parent owner

geothermal_power = geothermal_power[cols_to_keep]
geothermal_power.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
geothermal_power['sector'] = "geothermal power plant"

# look at the final set
# geothermal_power.columns
geothermal_power



# Oil and gas extraction

In [ ]:
# load data
oil_and_gas_extraction  = pd.read_excel(data_folder + dict_files['oil_and_gas_extraction']['file_name'], sheet_name = dict_files['oil_and_gas_extraction']['sheet_name'])

# rename columns
oil_and_gas_extraction.columns   = oil_and_gas_extraction.columns.str.lower().str.replace(" ", "_")

# # report pre-cleaning
print(f"Oil and gas extraction before cleaning dimensions: {oil_and_gas_extraction.shape}")

# # keep only operating plants
oil_and_gas_extraction = oil_and_gas_extraction[oil_and_gas_extraction.status == "operating"].reset_index()
print(f"Number of operating plants: {oil_and_gas_extraction.shape[0]}")

# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", "parent", 
                 "country", "region" , "latitude", "longitude", 
                 "accuracy", "start_year", "status", 'discovery_year', "last_updated"]

cols_to_keep = ['unit_name', 'owner', "parent", 
                'country', 'subnational_unit_(province,_state)', 'latitude', 'longitude', 
                'location_accuracy', 'production_start_year', 'status', 'discovery_year', 'status_year']
    # does not have info about parent location and capacity, and location of production

oil_and_gas_extraction = oil_and_gas_extraction[cols_to_keep]
oil_and_gas_extraction.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
oil_and_gas_extraction['sector'] = "oil and gas extraction"

# look at the final set
# oil_and_gas_extraction.columns
oil_and_gas_extraction



# Wind power

In [ ]:
# load data
wind_power              = pd.read_excel(data_folder + dict_files['wind_power']['file_name'], sheet_name = dict_files['wind_power']['sheet_name']) 

# rename columns
wind_power.columns   = wind_power.columns.str.lower().str.replace(" ", "_")

# # report pre-cleaning
print(f"Wind power data before cleaning dimensions: {wind_power.shape}")

# # keep only operating plants
wind_power = wind_power[wind_power.status == "operating"].reset_index()
print(f"Number of operating plants: {wind_power.shape[0]}")

# # convert terminal and unitname into one
wind_power['phase_name'].fillna(value = "", inplace = True)
wind_power['asset_name'] = wind_power.project_name + " " + wind_power.phase_name

# # keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", 
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status", "last_updated"]

cols_to_keep = ['asset_name', 'owner', 
                'capacity_(mw)', 'country', 'region', 'city', 'latitude', 'longitude', 
                'location_accuracy', 'start_year', 'status', 'date_last_researched']

wind_power = wind_power[cols_to_keep]
wind_power.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
wind_power['sector'] = "wind power plant"

# look at the final set
# wind_power.columns
wind_power



# Steel plant

In [ ]:
# load data
steel_plant             = pd.read_excel(data_folder + dict_files['steel_plant']['file_name'], sheet_name = dict_files['steel_plant']['sheet_name'])

# rename columns
steel_plant.columns   = steel_plant.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Steel plant data before cleaning dimensions: {steel_plant.shape}")

# # keep only operating plants
steel_plant = steel_plant[steel_plant.status == "operating"].reset_index()
print(f"Number of operating plants: {steel_plant.shape[0]}")

# split coordinates into latitude and longitude
steel_plant[['latitude', 'longitude']] = steel_plant['coordinates'].str.split(',', expand=True)
steel_plant.latitude = pd.to_numeric(steel_plant.latitude) 
steel_plant.longitude = pd.to_numeric(steel_plant.longitude) 

# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", "parent", 
                 "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "construction_year", "status"]

cols_to_keep = ['plant_name_(english)', 'owner', 'parent_[formula]', 
                'country', 'region', 'municipality', 'latitude', 'longitude', 
                'coordinate_accuracy', 'start_date', 'construction_date', 'status']


steel_plant = steel_plant[cols_to_keep]
steel_plant.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
steel_plant['sector'] = "steel plant"

# look at the final set
steel_plant.columns
steel_plant


# Coal

In [ ]:
# load data
coal_plant              = pd.read_excel(data_folder + dict_files['coal_plant']['file_name'], sheet_name = dict_files['coal_plant']['sheet_name'])

# rename columns
coal_plant.columns   = coal_plant.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Steel plant data before cleaning dimensions: {coal_plant.shape}")

# # keep only operating plants
coal_plant = coal_plant[coal_plant.status == "operating"].reset_index()
print(f"Number of operating plants: {coal_plant.shape[0]}")

# convert plant and unit name into one
coal_plant['unit_name'].fillna(value = "", inplace = True)
coal_plant['asset_name'] = coal_plant.plant_name + " " + coal_plant.unit_name


# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", "parent", 
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status"]

cols_to_keep = ['asset_name', 'owner', 'parent', 
                'capacity_(mw)', 'country', 'region', 'location', 'latitude', 'longitude', 
                'location_accuracy', 'start_year', 'status']

coal_plant = coal_plant[cols_to_keep]
coal_plant.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
coal_plant['sector'] = "coal plant"

# look at the final set
coal_plant.columns
coal_plant


# Solar power

In [ ]:
# load data
solar_power             = pd.read_excel(data_folder + dict_files['solar_power']['file_name'], sheet_name = dict_files['solar_power']['sheet_name'])

# rename columns
solar_power.columns   = solar_power.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Solar power plant data before cleaning dimensions: {solar_power.shape}")

# # keep only operating plants
solar_power = solar_power[solar_power.status == "operating"].reset_index()
print(f"Number of operating plants: {solar_power.shape[0]}")

# convert plant and unit name into one
solar_power['phase_name'].fillna(value = "", inplace = True)
solar_power['asset_name'] = solar_power.project_name + " " + solar_power.phase_name


# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", 
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status", "last_updated"]

cols_to_keep = ['asset_name', 'owner', 
                'capacity_(mw)', 'country', 'region', 'city', 'latitude', 'longitude', 
                'location_accuracy', 'start_year', 'status', 'date_last_researched']

solar_power = solar_power[cols_to_keep]
solar_power.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
solar_power['sector'] = "solar power plant"

# look at the final set
solar_power.columns
solar_power


# Bioenergy power

In [ ]:
# load data
bioenergy_power         = pd.read_excel(data_folder + dict_files['bioenergy_power']['file_name'], sheet_name = dict_files['bioenergy_power']['sheet_name'])

# rename columns
bioenergy_power.columns   = bioenergy_power.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Bioenergy power data before cleaning dimensions: {bioenergy_power.shape}")

# keep only operating plants
bioenergy_power = bioenergy_power[bioenergy_power.operating_status == "operating"].reset_index()
print(f"Number of operating plants: {bioenergy_power.shape[0]}")

# convert plant and unit name into one
bioenergy_power['unit_name'].fillna(value = "", inplace = True)
bioenergy_power['asset_name'] = bioenergy_power.project_name + " " + bioenergy_power.unit_name


# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", 
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status", "last_updated"]

cols_to_keep = ['asset_name', 'owner', 
                'capacity_(mw)', 'country', 'region', 'city', 'latitude', 'longitude', 
                'location_accuracy', 'unit_start_year', 'operating_status', 'date_last_researched']

bioenergy_power = bioenergy_power[cols_to_keep]
bioenergy_power.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
bioenergy_power['sector'] = "bioenergy power plant"

# # look at the final set
bioenergy_power.columns
bioenergy_power

# Coal terminal

In [ ]:
# load data
coal_terminals          = pd.read_excel(data_folder + dict_files['coal_terminals']['file_name'], sheet_name = dict_files['coal_terminals']['sheet_name'])

# rename columns
coal_terminals.columns   = coal_terminals.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Coal terminal data before cleaning dimensions: {coal_terminals.shape}")

# keep only operating plants
coal_terminals = coal_terminals[coal_terminals.status == "Operating"].reset_index()
print(f"Number of operating plants: {coal_terminals.shape[0]}")

# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", 
                 "capacity", "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status"]

cols_to_keep = ['coal_terminal_name', 'owner', 
                'capacity_(mt)', 'country', 'region', 'location', 'latitude', 'longitude', 
                'accuracy', 'opening_year', 'status']

coal_terminals = coal_terminals[cols_to_keep]
coal_terminals.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
coal_terminals['sector'] = "coal terminal plant"

# # look at the final set
coal_terminals.columns
coal_terminals

# Coal mine


In [ ]:
# load data
coal_mine               = pd.read_excel(data_folder + dict_files['coal_mine']['file_name'], sheet_name = dict_files['coal_mine']['sheet_name'])

# rename columns
coal_mine.columns   = coal_mine.columns.str.lower().str.replace(" ", "_")

# report pre-cleaning
print(f"Coal mines data before cleaning dimensions: {coal_mine.shape}")

# keep only operating plants
coal_mine = coal_mine[coal_mine.status == "Operating"].reset_index()
print(f"Number of operating plants: {coal_mine.shape[0]}")

# keep only relevant columns - MAKE SURE IT IS IN THE ORDER OF FEATURE NAMES!!!
feature_names = ["asset_name", "owner", "parent", 
                 "country", "region", "location" , "latitude", "longitude", 
                 "accuracy", "start_year", "status"]

cols_to_keep = ['mine_name', 'owners', 'parent_company',
                'country', 'region', 'location', 'latitude', 'longitude', 
                'location_accuracy', 'opening_year', 'status']

coal_mine = coal_mine[cols_to_keep]
coal_mine.rename(columns = {old: new for old, new in list(zip(cols_to_keep, feature_names))}, inplace = True)

# generate sector 
coal_mine['sector'] = "coal mine"

# # look at the final set
coal_mine.columns
coal_mine

# Combine all datasets and save as CSV

In [ ]:
df = pd.DataFrame()
i = 0

for file in sectors:
    df = pd.concat([df, globals()[file]], ignore_index=True)

df.to_csv('data/cleaned/gem_data.csv', index=False, sep='\t', encoding='utf-8')
df